In [1]:
!pip freeze > requirements.lock.txt

In [ ]:
!pip install corus
!pip install razdel
!pip install pymorphy2

In [3]:
import re
import nltk
import json
import pymorphy2
import pandas as pd
from zipfile import ZipFile
from corus import load_lenta
from textblob import TextBlob
from nltk.corpus import stopwords
from razdel import tokenize, sentenize

random_state = 9

### Импорт данных

In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [ ]:
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

In [ ]:
data = [next(records).text for i in range(1000)]
data[5]

### Предобработка

#### Нормализация и удаление стоп слов

In [ ]:
morph = pymorphy2.MorphAnalyzer()
nltk.download('stopwords')

def normalization(data, stopwords = stopwords.words('russian')):
  text = re.sub('ё', 'е', data.lower())
  text = re.sub(r'([.,!?])', r' \1', text)
  text = re.sub(r'[^а-яА-Я\s]+', '', text)
  text = text.strip()
  text = [w for w in text.split() if w not in stopwords]
  text = [w for w in text if len(w) >= 2]

  return ' '.join(text)

#### Лемматизация и токенизация

In [10]:
def lem_tok(data, morph = morph):
  result = [morph.parse(x)[0].normal_form for x in data.split(' ')]

  #result = ' '.join([morph.parse(x)[0].normal_form for x in data.split()]) #рез-т - строка
  #result = ' '.join([x for x in result.split() if x not in ['',]]) #добавить стоп-слова

  return result


### Окончательная обработка


In [11]:
def get_result(data, morph = morph, stopwords = stopwords.words('russian')):
  result = normalization(data)
  result = lem_tok(result)

  return result

In [12]:
data_processed = [get_result(elem) for elem in data]

In [ ]:
data_processed[1]

### Черновик

#### Новости на английском - вариант на потом


In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

In [ ]:
api_token = {"username":"w1nston","key":"47aa13e25604df989b986d1dd9300076"}

with open('/root/.kaggle/kaggle.json', 'w+') as file:
    json.dump(api_token, file)

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d rmisra/news-category-dataset

In [ ]:
with ZipFile('/content/news-category-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
base_file = '/content/News_Category_Dataset_v3.json'
df = pd.read_json(base_file, lines=True)
df.head()

In [ ]:
data = df.headline

#### Токенизация - старый вариант

In [ ]:
text = data[5]
text = [_.text for _ in list(sentenize(text))]
text

In [9]:
def tokenization(data):
    #на вход - преобработанный текст-строка, где слова разделены пробелами
    result = data.split(' ')

    return result